# tags of stack overflow questions

In [2]:
!pip install --upgrade tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.8 MB/s eta 0:00:00


In [3]:
import collections
import pathlib

import tensorflow as tf

from tensorflow.keras import layers, losses, utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [4]:
data_url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset_dir = utils.get_file(
    origin=data_url,
    untar=True,
    cache_dir="stach_overflow",
    cache_subdir=''
)

dataset_dir = pathlib.Path(dataset_dir).parent

6053168/6053168 [==============================] - 0s 0us/step


In [5]:
print(dataset_dir.iterdir())

<generator object Path.iterdir at 0x7f57a82294a0>


In [6]:
train_dir = dataset_dir/'train'
print(train_dir.iterdir())

<generator object Path.iterdir at 0x7f5721669dd0>


In [7]:
sample_file = train_dir/'python/1755.txt'

with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [8]:
batch_size = 32
seed = 42

raw_train_ds = utils.text_dataset_from_directory(  
    train_dir,
    batch_size=batch_size,
    validation_split=.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [9]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    print("Question: ", text_batch.numpy()[i])
    print("Label: ", label_batch.numpy()[i])

Question:  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the default con

In [10]:
for label, class_name in enumerate(raw_train_ds.class_names):
  print(f"Label {label} corresponds to {class_name}")

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [11]:
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [12]:
test_dir = dataset_dir/'test'
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


In [13]:
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary'
)

In [14]:
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

In [15]:
train_text = raw_train_ds.map(lambda text, label: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [16]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, axis=-1)
  return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
  text = tf.expand_dims(text, axis=-1)
  return int_vectorize_layer(text), label

In [17]:
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [18]:
print(f"'binary' vectorized question", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [19]:
print("'int' vectorized question", 
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question tf.Tensor(
[[ 55   6   2 410 211 229 121 895   4 124  32 245  43   5   1   1   5   1
    1   6   2 410 211 191 318  14   2  98  71 188   8   2 199  71 178   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 

In [20]:
print('1289th vocabulary: ', int_vectorize_layer.get_vocabulary()[1289])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289th vocabulary:  roman
Vocabulary size: 10000


In [21]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [22]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [23]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

In [24]:
binary_model = tf.keras.Sequential([layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10
)

Epoch 1/10
200/200 [==============================] - 11s 13ms/step - loss: 1.1227 - accuracy: 0.6408 - val_loss: 0.9163 - val_accuracy: 0.7794
Epoch 2/10
200/200 [==============================] - 1s 5ms/step - loss: 0.7809 - accuracy: 0.8202 - val_loss: 0.7516 - val_accuracy: 0.8037
Epoch 3/10
200/200 [==============================] - 2s 8ms/step - loss: 0.6289 - accuracy: 0.8616 - val_loss: 0.6656 - val_accuracy: 0.8169
Epoch 4/10
200/200 [==============================] - 2s 9ms/step - loss: 0.5351 - accuracy: 0.8873 - val_loss: 0.6119 - val_accuracy: 0.8238
Epoch 5/10
200/200 [==============================] - 1s 6ms/step - loss: 0.4689 - accuracy: 0.9028 - val_loss: 0.5752 - val_accuracy: 0.8331
Epoch 6/10
200/200 [==============================] - 1s 7ms/step - loss: 0.4185 - accuracy: 0.9159 - val_loss: 0.5484 - val_accuracy: 0.8356
Epoch 7/10
200/200 [==============================] - 1s 7ms/step - loss: 0.3781 - accuracy: 0.9275 - val_loss: 0.5283 - val_accuracy: 0.8369
Epoc

In [25]:
def  create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding='valid', activation='relu', strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [26]:
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 35s 145ms/step - loss: 1.1547 - accuracy: 0.5094 - val_loss: 0.7576 - val_accuracy: 0.6906
Epoch 2/5
200/200 [==============================] - 1s 5ms/step - loss: 0.6216 - accuracy: 0.7556 - val_loss: 0.5360 - val_accuracy: 0.8037
Epoch 3/5
200/200 [==============================] - 1s 5ms/step - loss: 0.3768 - accuracy: 0.8805 - val_loss: 0.4679 - val_accuracy: 0.8219
Epoch 4/5
200/200 [==============================] - 1s 5ms/step - loss: 0.2091 - accuracy: 0.9508 - val_loss: 0.4632 - val_accuracy: 0.8200
Epoch 5/5
200/200 [==============================] - 1s 5ms/step - loss: 0.1039 - accuracy: 0.9825 - val_loss: 0.4855 - val_accuracy: 0.8206


In [27]:
print("Linear model on binary data:")
print(binary_model.summary())

Linear model on binary data:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 40004     
                                                                 
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
print("Linear model on int data:")
print(int_model.summary())

Linear model on int data:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          640064    
                                                                 
 conv1d (Conv1D)             (None, None, 64)          20544     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 1s 5ms/step - loss: 0.5145 - accuracy: 0.8086
Binary model accuracy: 81.45%
Int model accuracy: 80.86%


In [30]:
export_model = tf.keras.Sequential([
    binary_vectorize_layer, binary_model,
    layers.Activation('sigmoid')
])

export_model.compile(loss=losses.SparseCategoricalCrossentropy(),
                     optimizer='adam',
                     metrics=['accuracy'])

loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {:2.2%}".format(accuracy))

250/250 [==============================] - 2s 7ms/step - loss: 0.5182 - accuracy: 0.8145
Accuracy: 81.45%


In [31]:
def get_string_labels(predicted_score_batch):
  predicted_int_labels = tf.math.argmax(predicted_score_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

In [32]:
inputs = [
    "how do I extract keys from a dict into a list?",  # 'python'
    "debug public static void main(string[] args) {...}",  # 'java'
]
predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)
for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("predicted label; ", label.numpy())

1/1 [==============================] - 0s 175ms/step
Question:  how do I extract keys from a dict into a list?
predicted label;  b'python'
Question:  debug public static void main(string[] args) {...}
predicted label;  b'java'


# predict the author of iliad translations

In [33]:
DIRECTORY_URL = "https://storage.googleapis.com/download.tensorflow.org/data/illiad/"
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
  text_dir = utils.get_file(name, origin=DIRECTORY_URL + name)

parent_dir = pathlib.Path(text_dir).parent
print(list(parent_dir.iterdir()))

807992/807992 [==============================] - 0s 0us/step
[PosixPath('/root/.keras/datasets/derby.txt'), PosixPath('/root/.keras/datasets/butler.txt'), PosixPath('/root/.keras/datasets/cowper.txt')]


In [34]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

In [35]:
labeled_data_sets = []

for i, file_name in enumerate(FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(str(parent_dir/file_name))
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

In [36]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [37]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False
)

In [38]:
for text, label in all_labeled_data.take(10):
  print("Sentence: ", text.numpy())
  print("Label: ", label.numpy())

Sentence:  b'Minerva gave up her seat to her, and she sat down by the side of father'
Label:  2
Sentence:  b'These two, of strength and life at once bereft,'
Label:  1
Sentence:  b'"O friends! the chiefs and councillors of Greece,'
Label:  1
Sentence:  b'So saying, illustrious Hector from the clash'
Label:  0
Sentence:  b'Let none among you, male or female, dare'
Label:  1
Sentence:  b'A pastured bullock smites behind the horns'
Label:  0
Sentence:  b'meet him. The lamentation of Andromache, Hecuba, and Helen, with the'
Label:  1
Sentence:  b'keep off both horse and foot, that the Trojan chieftains may not bear'
Label:  2
Sentence:  b'Again to battle mingling, with their hoofs'
Label:  0
Sentence:  b'hear of my being alive at the ships of the Achaeans."'
Label:  2


In [39]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [40]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [41]:
tokenized_ds = all_labeled_data.map(tokenize)

In [42]:
for text_batch in tokenized_ds.take(5):
  print("Tokens; ", text_batch.numpy())

Tokens;  [b'minerva' b'gave' b'up' b'her' b'seat' b'to' b'her' b',' b'and' b'she'
 b'sat' b'down' b'by' b'the' b'side' b'of' b'father']
Tokens;  [b'these' b'two' b',' b'of' b'strength' b'and' b'life' b'at' b'once'
 b'bereft' b',']
Tokens;  [b'"' b'o' b'friends' b'!' b'the' b'chiefs' b'and' b'councillors' b'of'
 b'greece' b',']
Tokens;  [b'so' b'saying' b',' b'illustrious' b'hector' b'from' b'the' b'clash']
Tokens;  [b'let' b'none' b'among' b'you' b',' b'male' b'or' b'female' b',' b'dare']


In [43]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  10000
First five vocab entries: [b',', b'the', b'and', b"'", b'of']


In [44]:
keys = vocab
values = range(2, len(vocab) + 2)

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64
)

num_oov_buckets = 1
vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov_buckets)

In [45]:
def preprocess_text(text, label):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

In [46]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence:  b'Minerva gave up her seat to her, and she sat down by the side of father'
Vectorized sentence:  [293 215 147  50 527   8  50   2   4  69 308 112  26   3 145   6 144]


In [47]:
all_encoded_data = all_labeled_data.map(preprocess_text)

In [48]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [49]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

In [50]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Text batch shape:  (64, 18)
Label batch shape:  (64,)
First text example:  tf.Tensor([293 215 147  50 527   8  50   2   4  69 308 112  26   3 145   6 144   0], shape=(18,), dtype=int64)
First label example:  tf.Tensor(2, shape=(), dtype=int64)


In [51]:
vocab_size += 2

In [52]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

In [53]:
model = create_model(vocab_size=vocab_size, num_labels=3)

model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

history = model.fit(train_data, validation_data=validation_data, epochs=3)

Epoch 1/3
697/697 [==============================] - 95s 55ms/step - loss: 0.5203 - accuracy: 0.7677 - val_loss: 0.3774 - val_accuracy: 0.8452
Epoch 2/3
697/697 [==============================] - 3s 4ms/step - loss: 0.2854 - accuracy: 0.8839 - val_loss: 0.3647 - val_accuracy: 0.8504
Epoch 3/3
697/697 [==============================] - 4s 6ms/step - loss: 0.1924 - accuracy: 0.9280 - val_loss: 0.3977 - val_accuracy: 0.8418


In [54]:
loss, accuracy = model.evaluate(validation_data)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 0s 3ms/step - loss: 0.3977 - accuracy: 0.8418
Loss:  0.3977464437484741
Accuracy: 84.18%


In [55]:
preprocess_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)


preprocess_layer.set_vocabulary(vocab)

In [56]:
export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')]
)

export_model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=False),
                     optimizer='adam',
                     metrics=['accuracy'])

In [57]:
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)

loss, accuracy = export_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

79/79 [==============================] - 9s 23ms/step - loss: 0.5574 - accuracy: 0.7852
Loss:  0.5573740005493164
Accuracy: 78.52%


In [58]:
inputs = [
    "Join'd to th' Ionians with their flowing robes,",  # Label: 1
    "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
    "And with loud clangor of his arms he fell.",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.math.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

1/1 [==============================] - 3s 3s/step
Question:  Join'd to th' Ionians with their flowing robes,
Predicted label:  1
Question:  the allies, and his armour flashed about him so that he seemed to all
Predicted label:  2
Question:  And with loud clangor of his arms he fell.
Predicted label:  0


# tfds imdb movie reviews

In [59]:
train_ds = tfds.load(
    'imdb_reviews',
    split='train[:80%]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYTZ0VD/imdb_reviews-train.tfrecord…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYTZ0VD/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteYTZ0VD/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [60]:
val_ds = tfds.load(
    'imdb_reviews',
    split='train[80%:]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True
)

In [61]:
for review_batch, label_batch in val_ds.take(1):
  for i in range(5):
    print("Review: ", review_batch[i].numpy())
    print("Label: ", label_batch[i].numpy())

Review:  b"Instead, go to the zoo, buy some peanuts and feed 'em to the monkeys. Monkeys are funny. People with amnesia who don't say much, just sit there with vacant eyes are not all that funny.<br /><br />Black comedy? There isn't a black person in it, and there isn't one funny thing in it either.<br /><br />Walmart buys these things up somehow and puts them on their dollar rack. It's labeled Unrated. I think they took out the topless scene. They may have taken out other stuff too, who knows? All we know is that whatever they took out, isn't there any more.<br /><br />The acting seemed OK to me. There's a lot of unfathomables tho. It's supposed to be a city? It's supposed to be a big lake? If it's so hot in the church people are fanning themselves, why are they all wearing coats?"
Label:  0
Review:  b'Well, was Morgan Freeman any more unusual as God than George Burns? This film sure was better than that bore, "Oh, God". I was totally engrossed and LMAO all the way through. Carrey was

In [62]:
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

train_text = train_ds.map(lambda text, label: text)
vectorize_layer.adapt(train_text)

In [63]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, axis=-1)
  return vectorize_layer(text), label

In [64]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [71]:
train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

In [72]:
model = create_model(vocab_size = VOCAB_SIZE + 1, num_labels=1)
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 64)          640064    
                                                                 
 conv1d_3 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 660,673
Trainable params: 660,673
Non-trainable params: 0
_________________________________________________________________


In [73]:
model.compile(optimizer='adam',
              loss=losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [74]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
313/313 [==============================] - 28s 87ms/step - loss: 0.5408 - accuracy: 0.6640 - val_loss: 0.3793 - val_accuracy: 0.8262
Epoch 2/3
313/313 [==============================] - 2s 6ms/step - loss: 0.3043 - accuracy: 0.8668 - val_loss: 0.3233 - val_accuracy: 0.8582
Epoch 3/3
313/313 [==============================] - 2s 7ms/step - loss: 0.1886 - accuracy: 0.9263 - val_loss: 0.3306 - val_accuracy: 0.8602


In [75]:
loss, accurcy = model.evaluate(val_ds)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy:2.2%}")

79/79 [==============================] - 0s 5ms/step - loss: 0.3306 - accuracy: 0.8602
Loss: 0.3305864632129669
Accuracy: 78.52%


In [76]:
export_model = tf.keras.Sequential([
    vectorize_layer, model,
    layers.Activation('sigmoid')
])

export_model.compile(
    
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy']
)

In [77]:
inputs = [
    "This is a fantastic movie.",
    "This is a bad movie.",
    "This movie was so bad that it was good.",
    "I will never say yes to watching this movie.",
]

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]

for input, label in zip(inputs, predicted_labels):
  print("Question: ", inputs)
  print("Predicted label: ", label)

1/1 [==============================] - 0s 224ms/step
Question:  ['This is a fantastic movie.', 'This is a bad movie.', 'This movie was so bad that it was good.', 'I will never say yes to watching this movie.']
Predicted label:  1
Question:  ['This is a fantastic movie.', 'This is a bad movie.', 'This movie was so bad that it was good.', 'I will never say yes to watching this movie.']
Predicted label:  0
Question:  ['This is a fantastic movie.', 'This is a bad movie.', 'This movie was so bad that it was good.', 'I will never say yes to watching this movie.']
Predicted label:  0
Question:  ['This is a fantastic movie.', 'This is a bad movie.', 'This movie was so bad that it was good.', 'I will never say yes to watching this movie.']
Predicted label:  0
